In [1]:
import csv
import pandas as pd
from itertools import islice
import h5py
import numpy as np
from scipy import sparse
import re
import scanpy as sc
from preprocessing_function import *

/tmp/ipykernel_75764/1059863759.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Choices

In [2]:
#########################
# selection of the data #
#########################

type_data = 'enhancer'
# type_data = 'promotor'
# type_data = 'enhancer_promotor'


data_path = f'../../../results_scbasset/{type_data}/'

#########
# Input #
#########

csv_file = f'../../../results_scbasset/data/CAGE/{type_data}.csv'

##########
# Output #
##########

output_csv = f'{data_path}/data/peaks_{type_data}.bed'
output_count_matrix = f'{data_path}data/count_matrix_{type_data}.h5'



# Import data

In [3]:
data = pd.read_csv(csv_file, sep = ',', header = 0)

## Completing the chromosomal coordinates

In [5]:
df_adjust_coordinates = adjust_coordinates(data)

# Create bed file (peaks coordinates)

In [6]:
df_peak_position = df_adjust_coordinates['adjusted_coordinate']

create_bed_file(df_peak_position, output_csv)

Successfully created BED file: ../../../results_scbasset/enhancer//data/peaks_enhancer.bed


# Create count matrix H5 file

## Extract the usefull informations

In [7]:
barcodes_list, data_matrix_transposed, features_list, feature_types_data, genome_data = extract_info(data, df_peak_position)

Number of barcodes (patients):  98
Number of peaks:  104819


## Create the h5 file

In [8]:
create_count_matrix_h5(barcodes_list, data_matrix_transposed, features_list, feature_types_data, genome_data, output_count_matrix)

Successfully created ../../../results_scbasset/enhancer/data/count_matrix_enhancer.h5 file


In [12]:
Visualize_h5_file('../../../results_scbasset/enhancer/data/count_matrix_enhancer.h5', 'matrix')

Keys available in the HDF5 file: ['matrix']

Dimension: (98,)
Value of dataset 'barcodes': [b's15t1p2sq1' b's15t2p2sq1' b's15t3p2sq1' b's15t4p2sq1' b's15t5p2sq1'
 b's15t6p2sq1' b's15t7p2sq1' b's15t8p2sq1' b's1t1p1sq1' b's1t2p1sq1'
 b's1t3p1sq1' b's1t4p1sq1' b's1t5p2sq1' b's1t6p2sq1' b's1t7p2sq1'
 b's1t8p2sq1' b's7t1p1sq1' b's7t2p1sq1' b's7t3p1sq1' b's7t4p1sq1'
 b's7t5p1sq1' b's7t6p1sq1' b's7t7p1sq1' b's7t8p1sq1' b's2t1p3sq2'
 b's2t2p3sq2' b's2t3p3sq2' b's2t4p3sq2' b's2t5p3sq2' b's2t6p3sq2'
 b's2t7p3sq2' b's2t8p3sq2' b's3t1p3sq2' b's3t2p3sq2' b's3t3p3sq2'
 b's3t4p3sq2' b's3t5p4sq2' b's3t6p4sq2' b's3t7p4sq2' b's3t8p4sq2'
 b's4t1p4sq2' b's4t2p4sq2' b's4t3p4sq2' b's4t4p4sq2' b's4t5p4sq2'
 b's4t6p4sq2' b's4t7p4sq2' b's4t8p4sq2' b's5t1p5sq3' b's5t2p5sq3'
 b's5t3p5sq3' b's5t4p5sq3' b's5t5p5sq3' b's5t6p5sq3' b's5t7p5sq3'
 b's5t8p5sq3' b's6t1p5sq3' b's6t2p5sq3' b's6t3p5sq3' b's6t4p5sq3'
 b's6t5p6sq3' b's6t6p6sq3' b's6t7p6sq3' b's6t8p6sq3' b's9t1p6sq3'
 b's9t2p6sq3' b's9t3p6sq3' b's9t4p6sq3' b's